In [1]:
import sys
from pathlib import Path

# Add src directory to Python path
project_root = Path.cwd().parent  
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to Python path")


Added /Users/cooper/Desktop/hydro-forecasting/src to Python path


In [2]:
from hydro_forecasting.data.lazy_datamodule import HydroLazyDataModule
from hydro_forecasting.preprocessing.grouped import GroupedPipeline
from sklearn.pipeline import Pipeline
from hydro_forecasting.preprocessing.standard_scale import StandardScaleTransformer
from hydro_forecasting.data.caravanify_parquet import (
    CaravanifyParquet,
    CaravanifyParquetConfig,
)


---

In [3]:
config = CaravanifyParquetConfig(
    attributes_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes",
    timeseries_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv",
    shapefile_dir="/Users/cooper/Desktop/CAMELS-CH/data/CARAVANIFY/CA/post_processed/shapefiles",
    gauge_id_prefix="CA",
    use_hydroatlas_attributes=True,
    use_caravan_attributes=True,
    use_other_attributes=True,
)

caravan = CaravanifyParquet(config)
basin_ids = caravan.get_all_gauge_ids()[:3]

In [4]:
forcing_features = [
    "snow_depth_water_equivalent_mean",
    "surface_net_solar_radiation_mean",
    "surface_net_thermal_radiation_mean",
    "potential_evaporation_sum_ERA5_LAND",
    "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
    "temperature_2m_mean",
    "temperature_2m_min",
    "temperature_2m_max",
    "total_precipitation_sum",
]

static_features = [
    # "gauge_id",
    "p_mean",
    "area",
    "ele_mt_sav",
    "high_prec_dur",
    "frac_snow",
    "high_prec_freq",
    "slp_dg_sav",
    "cly_pc_sav",
    "aridity_ERA5_LAND",
    "aridity_FAO_PM",
]

In [5]:
feature_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer())]),
    columns=forcing_features,
    group_identifier="gauge_id",
)

target_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer())]),
    columns=["streamflow"],
    group_identifier="gauge_id",
)

static_pipeline = Pipeline([("scaler", StandardScaleTransformer())])

preprocessing_config = {
    "features": {"pipeline": feature_pipeline},
    "target": {"pipeline": target_pipeline},
    "static_features": {"pipeline": static_pipeline, "columns": static_features},
}

In [6]:
datamodule = HydroLazyDataModule(
    path_to_time_series_directory="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv/CA",
    path_to_static_attributes_directory="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA",
    path_to_preprocessing_output_directory="/Users/cooper/Desktop/hydro-forecasting/tests/yolo_5",
    group_identifier="gauge_id",
    batch_size=2048,
    input_length=70,
    output_length=10,
    forcing_features=forcing_features,
    static_features=static_features,
    target="streamflow",
    preprocessing_configs=preprocessing_config,
    num_workers=4,
    min_train_years=5,
    train_prop=0.5,
    val_prop=0.25,
    test_prop=0.25,
    max_imputation_gap_size=5,
    list_of_gauge_ids_to_process=basin_ids,
    is_autoregressive=True
)


In [7]:
datamodule.prepare_data()

Found 3 basin files
Loading caravan attributes from /Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA/attributes_caravan_CA.parquet
Loading hydroatlas attributes from /Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA/attributes_hydroatlas_CA.parquet
Loading other attributes from /Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA/attributes_other_CA.parquet
Merging 3 attribute DataFrames
Loaded static attributes for 3 basins
Fitting preprocessing pipelines on all basins...


Loading basin data for fitting: 100%|██████████| 78/78 [00:00<00:00, 280.58it/s]


Found following uniquwe columns in sample data: ['date', 'snow_depth_water_equivalent_mean', 'surface_net_solar_radiation_mean', 'surface_net_thermal_radiation_mean', 'surface_pressure_mean', 'temperature_2m_mean', 'dewpoint_temperature_2m_mean', 'u_component_of_wind_10m_mean', 'v_component_of_wind_10m_mean', 'volumetric_soil_water_layer_1_mean', 'volumetric_soil_water_layer_2_mean', 'volumetric_soil_water_layer_3_mean', 'volumetric_soil_water_layer_4_mean', 'snow_depth_water_equivalent_min', 'surface_net_solar_radiation_min', 'surface_net_thermal_radiation_min', 'surface_pressure_min', 'temperature_2m_min', 'dewpoint_temperature_2m_min', 'u_component_of_wind_10m_min', 'v_component_of_wind_10m_min', 'volumetric_soil_water_layer_1_min', 'volumetric_soil_water_layer_2_min', 'volumetric_soil_water_layer_3_min', 'volumetric_soil_water_layer_4_min', 'snow_depth_water_equivalent_max', 'surface_net_solar_radiation_max', 'surface_net_thermal_radiation_max', 'surface_pressure_max', 'temperature

Processing basins: 100%|██████████| 3/3 [00:00<00:00,  4.47it/s]

Processing static attributes...
Saved transformed static attributes for 3 basins to /Users/cooper/Desktop/hydro-forecasting/tests/yolo_5/processed_static_data/static_attributes.parquet
Processing complete. 3 basins retained out of 3.


In [8]:
datamodule.setup(stage="fit")

DEBUG - Length of batch_index_entries: 8979
DEBUG - Length of batch_index_entries: 4558
Created training dataset with 8979 samples
Created validation dataset with 4558 samples


In [9]:
len(datamodule.index_entries_by_stage["train"]), len(datamodule.index_entries_by_stage["val"]), len(datamodule.index_entries_by_stage["test"])

(8979, 4558, 4438)